In [1]:
 ##Dataset final
import pandas as pd
import numpy as np

In [2]:
x_training_features=pd.read_csv("training_set_features_eda_notnulls.csv")
y_training_features=pd.read_csv("training_set_labels.csv")
#Los id no los necesitaremos al entrenar el modelo
x_training_features.drop(["Unnamed: 0.1","Unnamed: 0"], axis=1, inplace=True)

#Tomando en cuenta los features que están más correlacionados, se eliminarán behavioral_outside_home
# y doctor_recc_h1n1, tomando en cuenta tambien que tienen una menor correlación con las variables 
#targets frente a los features con los que se correlacionan.
x_training_features.drop(["behavioral_outside_home","doctor_recc_h1n1"], axis=1, inplace=True)

#Ahora solo necesitamos de las variables targets eliminar la columna que no necesitamos: los id
y_training_features.drop("respondent_id", axis=1, inplace=True)

In [3]:

from sklearn import preprocessing
l_encoder = preprocessing.LabelEncoder()

list_labels_encoders = ["age_group","education"]
list_dummies=["race","sex","marital_status","rent_or_own","employment_status","census_msa"]

# Se aplica label encoder a las variables ordinales
x_training_features["age_group"]=l_encoder.fit_transform(x_training_features["age_group"]) 
x_training_features["education"]=l_encoder.fit_transform(x_training_features["education"]) 

#se aplica one hot enocoder usando pandas: "Dummies"
x_training_features=pd.get_dummies(x_training_features, columns=list_dummies, prefix_sep='_')

In [5]:
#para evitar colinealidad debido a los dummies, se eliminará 1 columna por cada variable
#las columnas generadas por one hot encoder "dummies" que se eliminarán son:
list_dummies_drop=['race_Other or Multiple','sex_Male', \
            'marital_status_Not Married','rent_or_own_Own','employment_status_Unemployed', \
            'census_msa_Non-MSA']
x_training_features.drop(list_dummies_drop,axis=1, inplace=True)
x_training_features.columns

Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_touch_face',
       'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months',
       'health_worker', 'health_insurance', 'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective', 'opinion_seas_risk',
       'opinion_seas_sick_from_vacc', 'age_group', 'education',
       'household_adults', 'household_children', 'race_Black', 'race_Hispanic',
       'race_White', 'sex_Female', 'marital_status_Married',
       'rent_or_own_Rent', 'employment_status_Employed',
       'employment_status_Not in Labor Force',
       'census_msa_MSA, Not Principle  City',
       'census_msa_MSA, Principle City'],
      dtype='object')

In [6]:
#Analizando presencia de valores atípicos
def find_outliers(df):
    outliers = {}
    for columna in df.columns:
        # Calcular el primer y tercer cuartil
        Q1 = df[columna].quantile(0.25)
        Q3 = df[columna].quantile(0.75)
        # Calcular el rango intercuartílico (IQR)
        IQR = Q3 - Q1
        # Definir los límites para detectar outliers
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        # Encontrar los índices de los outliers
        outliers[columna] = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)].index.tolist()
        print(columna, "  , Cantidad de outliers: ", len(outliers[columna]))
    return outliers

    
outliers_per_feature = find_outliers(x_training_features)


h1n1_concern   , Cantidad de outliers:  0
h1n1_knowledge   , Cantidad de outliers:  0
behavioral_antiviral_meds   , Cantidad de outliers:  1301
behavioral_avoidance   , Cantidad de outliers:  0
behavioral_face_mask   , Cantidad de outliers:  1841
behavioral_wash_hands   , Cantidad de outliers:  4650
behavioral_large_gatherings   , Cantidad de outliers:  0
behavioral_touch_face   , Cantidad de outliers:  0
doctor_recc_seasonal   , Cantidad de outliers:  0
chronic_med_condition   , Cantidad de outliers:  0
child_under_6_months   , Cantidad de outliers:  2138
health_worker   , Cantidad de outliers:  2899
health_insurance   , Cantidad de outliers:  0
opinion_h1n1_vacc_effective   , Cantidad de outliers:  0
opinion_h1n1_risk   , Cantidad de outliers:  0
opinion_h1n1_sick_from_vacc   , Cantidad de outliers:  0
opinion_seas_vacc_effective   , Cantidad de outliers:  3427
opinion_seas_risk   , Cantidad de outliers:  0
opinion_seas_sick_from_vacc   , Cantidad de outliers:  6573
age_group   , Can

In [7]:
# Se realizarán dos dataframes, uno con los outliers y el otro sin ellos
#eliminando valores atipicos de la variable con más atipicos: opinion_seas_sick_from_vacc
#se espera posteriormente ver la posibilidad de mejora en el score de lo modelos al disminuir
#la cantidad de outliers
x_training_features_no_outliers = \
x_training_features.drop(outliers_per_feature["opinion_seas_sick_from_vacc"],axis=0)
find_outliers(x_training_features_no_outliers)
x_training_features_no_outliers.shape #20134 es la longitud

#los índices donde estaban los valores atípicos se utilizan para eliminarlos también 
#en el dataframe de la variable target
y_training_features_no_outliers = \
y_training_features.drop(outliers_per_feature["opinion_seas_sick_from_vacc"],axis=0)

#verificando que los dataframes tienen el mismo tamaño
x_training_features_no_outliers.shape[0] == y_training_features_no_outliers.shape[0]

h1n1_concern   , Cantidad de outliers:  0
h1n1_knowledge   , Cantidad de outliers:  0
behavioral_antiviral_meds   , Cantidad de outliers:  803
behavioral_avoidance   , Cantidad de outliers:  0
behavioral_face_mask   , Cantidad de outliers:  1161
behavioral_wash_hands   , Cantidad de outliers:  3825
behavioral_large_gatherings   , Cantidad de outliers:  0
behavioral_touch_face   , Cantidad de outliers:  0
doctor_recc_seasonal   , Cantidad de outliers:  0
chronic_med_condition   , Cantidad de outliers:  0
child_under_6_months   , Cantidad de outliers:  1487
health_worker   , Cantidad de outliers:  2199
health_insurance   , Cantidad de outliers:  0
opinion_h1n1_vacc_effective   , Cantidad de outliers:  0
opinion_h1n1_risk   , Cantidad de outliers:  0
opinion_h1n1_sick_from_vacc   , Cantidad de outliers:  3808
opinion_seas_vacc_effective   , Cantidad de outliers:  2521
opinion_seas_risk   , Cantidad de outliers:  0
opinion_seas_sick_from_vacc   , Cantidad de outliers:  0
age_group   , Cant

True

In [8]:
print("df con menos outliers: ",x_training_features_no_outliers.shape)
print("df con outliers: ",x_training_features.shape)

df con menos outliers:  (20134, 33)
df con outliers:  (26707, 33)


In [9]:
########################separando el train y test dataframe###############################
# ESTO SOLO SE HACE A MODO EJEMPLO, YA QUE LUEGO STRATIFIEDKFOLD SE APLICA DE MANERA CONJUNTA
#CON GRIDSERACH CV, ESTO SOLO PARA PRÓPOSITOS DE VERIFICAR SI ayuda a garantizar que cada fold 
#tenga una representación equitativa de todas las clases, evitando así la posibilidad de que un 
#fold tenga muy pocos ejemplos de una clase en particular.


#Ya que tenemos dos variables targets, haremos este proceso para cada una
y1=np.array(y_training_features['h1n1_vaccine'])
y2=np.array(y_training_features['seasonal_vaccine'])




from sklearn.model_selection import StratifiedKFold

############################## h1n1_vaccine ######################################
#splitting into 5 folds
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
train_list_index, test_list_index=[],[]
for i,(train_index, test_index) in enumerate(skf.split(x_training_features, y1)):
 print(f"Fold {i}:")
 print(f"  Train: index={train_index}")
 print(f"  Test:  index={test_index}")
 train_list_index.append(train_index) #saving the indexes od the 5 folds
 test_list_index.append(test_index)


Fold 0:
  Train: index=[    0     1     2 ... 26703 26704 26705]
  Test:  index=[    4     8    12 ... 26696 26701 26706]
Fold 1:
  Train: index=[    0     1     4 ... 26704 26705 26706]
  Test:  index=[    2     3     5 ... 26699 26700 26702]
Fold 2:
  Train: index=[    2     3     4 ... 26702 26705 26706]
  Test:  index=[    0     1     9 ... 26693 26703 26704]
Fold 3:
  Train: index=[    0     1     2 ... 26704 26705 26706]
  Test:  index=[    6    11    13 ... 26691 26695 26697]
Fold 4:
  Train: index=[    0     1     2 ... 26703 26704 26706]
  Test:  index=[    7    14    23 ... 26681 26692 26705]


In [9]:
############################## seasonal_vaccine ######################################
#splitting into 5 folds
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
train_list_index_2, test_list_index_2=[],[]
for i,(train_index, test_index) in enumerate(skf.split(x_training_features, y2)):
 print(f"Fold {i}:")
 print(f"  Train: index={train_index}")
 print(f"  Test:  index={test_index}")
 train_list_index_2.append(train_index) #saving the indexes od the 5 folds
 test_list_index_2.append(test_index)

Fold 0:
  Train: index=[    0     1     2 ... 26702 26705 26706]
  Test:  index=[    5     9    14 ... 26700 26703 26704]
Fold 1:
  Train: index=[    0     1     3 ... 26704 26705 26706]
  Test:  index=[    2    12    13 ... 26685 26694 26698]
Fold 2:
  Train: index=[    0     2     3 ... 26704 26705 26706]
  Test:  index=[    1     8    10 ... 26695 26699 26702]
Fold 3:
  Train: index=[    0     1     2 ... 26702 26703 26704]
  Test:  index=[    4    15    29 ... 26690 26705 26706]
Fold 4:
  Train: index=[    1     2     4 ... 26704 26705 26706]
  Test:  index=[    0     3     6 ... 26687 26696 26701]


In [10]:
#aplicando Standard Scaler to the 5 folds
from sklearn.preprocessing import StandardScaler
################################# h1n1_vaccine #######################################
x_train,x_test,y_train,y_test=[],[],[],[]
sc=StandardScaler()

for i in range(0, len(train_list_index)):
 x_train.append(sc.fit_transform(x_training_features.iloc[train_list_index[i]]))
 x_test.append(sc.transform(x_training_features.iloc[test_list_index[i]]))
 #target variable without standard scaler
 y_train.append(y_training_features.iloc[train_list_index[i]])
 y_test.append(y_training_features.iloc[test_list_index[i]])

In [11]:
#aplicando Standard Scaler to the 5 folds
################################# seasonal_vaccine #######################################
x_train_2,x_test_2,y_train_2,y_test_2=[],[],[],[]
sc=StandardScaler()

for i in range(0, len(train_list_index)):
 x_train_2.append(sc.fit_transform(x_training_features.iloc[train_list_index_2[i]]))
 x_test_2.append(sc.transform(x_training_features.iloc[test_list_index_2[i]]))
 #target variable without standard scaler
 y_train_2.append(y_training_features.iloc[train_list_index_2[i]])
 y_test_2.append(y_training_features.iloc[test_list_index_2[i]])

In [13]:
################INICIO DE DEFINICIÓN DE PARÁMETROS DE LOS MODELOS CON GRID SERCH CV#####################
#se harán dos modelos, uno para cada variable target

############################# y1=H1N1 variable target################################
from sklearn.linear_model import RidgeClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from numpy import logspace
from sklearn.preprocessing import StandardScaler

# Definir tus datos x_train y y_train

lambdas = logspace(-11, -2, 10)
C_ = [0.01, 0.1, 1, 10]  # How many errors can admit SVM
gamma_ = [0.01, 0.1, 1, 10]  # How curved the decision boundaries will be
alpha_ = [0.01, 0.1, 1, 10]  # Naive Bayes

# Explanation of alpha hyperparameter:
# Consider W* word not present in the training set
# P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
#  = Number of training points such that w* word present and Y=1 / Number of training points where Y=1
#  = 0/Number of training points where Y=1
# So to get rid of this problem we do Laplace smoothing. we add alpha to numerator and denominator field.
# = 0 + alpha / Number of training points where Y=1 + (Number of class labels in cl

estimators = [
    ('ridge', RidgeClassifierCV()),
    ('tree', DecisionTreeClassifier(splitter="best")),
    ('svm', SVC()),
    ('nv', BernoulliNB())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "tree": {"tree__max_depth": [5, 7, 10], "tree__ccp_alpha": [0.1, 0.25, 0.5], "tree__max_features": [10, 15, 20]},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf", "linear"]},
    "nv": {"nv__alpha": alpha_}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y1)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 32 candidates, totalling 96 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


In [15]:
#Selecting the best model
lead_board = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board =lead_board.sort_values(by="score", ascending=False)
#l_board.to_csv("lead_board_h1n1.csv")

In [18]:
l_board

,model,params,score
2,svm,"{'svm__C': 0.1, 'svm__gamma': 0.01, 'svm__kern...",0.823433
0,ridge,{'ridge__alphas': 0.001},0.821022
3,nv,{'nv__alpha': 1},0.786553
1,tree,"{'tree__ccp_alpha': 0.1, 'tree__max_depth': 5,...",0.500000


In [19]:
############################# y2=seasonal variable target################################


lambdas = logspace(-11, -2, 10)
C_ = [0.01, 0.1, 1, 10]  # How many errors can admit SVM
gamma_ = [0.01, 0.1, 1, 10]  # How curved the decision boundaries will be
alpha_ = [0.01, 0.1, 1, 10]  # Naive Bayes

# Explanation of alpha hyperparameter:
# Consider W* word not present in the training set
# P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
#  = Number of training points such that w* word present and Y=1 / Number of training points where Y=1
#  = 0/Number of training points where Y=1
# So to get rid of this problem we do Laplace smoothing. we add alpha to numerator and denominator field.
# = 0 + alpha / Number of training points where Y=1 + (Number of class labels in cl

estimators = [
    ('ridge', RidgeClassifierCV()),
    ('tree', DecisionTreeClassifier(splitter="best")),
    ('svm', SVC()),
    ('nv', BernoulliNB())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "tree": {"tree__max_depth": [5, 7, 10], "tree__ccp_alpha": [0.1, 0.25, 0.5], "tree__max_features": [10, 15, 20]},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf", "linear"]},
    "nv": {"nv__alpha": alpha_}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y2)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 32 candidates, totalling 96 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


In [22]:
#Selecting the best model
lead_board_2 = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board_2 =lead_board_2.sort_values(by="score", ascending=False)
#l_board_2.to_csv("lead_board_seasonal.csv")

In [24]:
l_board_2

,model,params,score
2,svm,"{'svm__C': 1, 'svm__gamma': 0.01, 'svm__kernel...",0.854143
0,ridge,{'ridge__alphas': 1e-08},0.847609
3,nv,{'nv__alpha': 0.01},0.818433
1,tree,"{'tree__ccp_alpha': 0.1, 'tree__max_depth': 5,...",0.500000


In [34]:
a=lead_board_2.loc[2]
a["params"]

{'svm__C': 1, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}

In [64]:
############################# y1=H1N1 variable target################################
###APLICANDO DE NUEVO GRID SEARCH CV, PERO AHORA SOLO PARA AFINAR (SOLO VARIANDO 
#LOs HIPERPARÁMETROS DE REGRESIÓN).
#Y VERIFICAR SI SE PUEDE APLICAR EL MISMO MODELO PARA AMBAS VARIABLES TARGETS

C_ = [0.1]  # How many errors can admit SVM
gamma_ = [0.001]  # How curved the decision boundaries will be
lambdas = [0.001, 1e-08]


estimators = [
    ('ridge', RidgeClassifierCV()),
    ('svm', SVC())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf"]}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y1)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [72]:
#Selecting the best model
lead_board_3 = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board_3 =lead_board_3.sort_values(by="score", ascending=False)
#l_board_3.to_csv("svm_board_h1n1.csv")

In [73]:
l_board_3

,model,params,score
0,ridge,{'ridge__alphas': 0.001},0.847579
1,svm,"{'svm__C': 0.1, 'svm__gamma': 0.001, 'svm__ker...",0.844966


In [71]:
############################# y2=seasonal variable target################################
###APLICANDO DE NUEVO GRID SEARCH CV, PERO AHORA SOLO PARA AFINAR (SOLO VARIANDO 
#LOs HIPERPARÁMETROS DE REGRESIÓN).
#Y VERIFICAR SI SE PUEDE APLICAR EL MISMO MODELO PARA AMBAS VARIABLES TARGETS


C_ = [0.1]  # How many errors can admit SVM
gamma_ = [0.001]  # How curved the decision boundaries will be
lambdas = [0.001, 1e-08]
# Explanation of alpha hyperparameter:
# Consider W* word not present in the training set
# P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
#  = Number of training points such that w* word present and Y=1 / Number of training points where Y=1
#  = 0/Number of training points where Y=1
# So to get rid of this problem we do Laplace smoothing. we add alpha to numerator and denominator field.
# = 0 + alpha / Number of training points where Y=1 + (Number of class labels in cl

estimators = [
    ('ridge', RidgeClassifierCV()),
    ('svm', SVC())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf"]}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y2)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [69]:
#Selecting the best model
lead_board_4 = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board_4 =lead_board_4.sort_values(by="score", ascending=False)
#l_board_4.to_csv("svm_board_seasonal.csv")

In [70]:
l_board_4

,model,params,score
1,svm,"{'svm__C': 0.1, 'svm__gamma': 0.001, 'svm__ker...",0.821622
0,ridge,{'ridge__alphas': 0.001},0.820940


In [ ]:
#SE OBSERVA QUE PARA H1N1 VARIABLE TARGET ES MEJOR ridge	{'ridge__alphas': 0.001}
#SE OBSERVA QUE PARA SEASONAL VARIABLE TARGET ES MEJOR svm	{'svm__C': 0.1, 'svm__gamma': 0.001,}
#POR ENDE SE DETERMINA USAR DIFERENTES MODELOS

In [85]:
#######Determinando el mejor learning rate previo ha aplicar adaboost, usando h1n1 target#### 

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define los valores de learning_rate a probar
param_grid = {'clf__learning_rate': [0.1, 0.2]}

# Crea un estimador AdaBoostClassifier con LogisticRegression y SVC como clasificadores base
logistic_base = LogisticRegression(C=0.001, solver='lbfgs', max_iter=1000)
svm_base = SVC(C=0.1, gamma=0.001, probability=True)

clf = AdaBoostClassifier(base_estimator=logistic_base, n_estimators=100)

# Crea un pipeline con StandardScaler y el clasificador
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', clf)
])

# Define StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Realiza la búsqueda en rejilla con StratifiedKFold y roc_auc como métrica
grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_kfold, scoring='roc_auc')
grid_search.fit(x_training_features, y1)

# El mejor valor de learning_rate
best_learning_rate = grid_search.best_params_['clf__learning_rate']

C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-p

In [86]:
best_learning_rate
#es 0.2

0.2

In [90]:
###########################Aplicación de adaboost, usando h1n1 target###############
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import pandas as pd

# Define tu learning_rate
learning_rate = 0.2

# Crea un estimador AdaBoostClassifier con LogisticRegression y SVC como clasificadores base
logistic_base = LogisticRegression(C=0.001, solver='lbfgs', max_iter=1000)
svm_base = SVC(C=0.1, gamma=0.001, probability=True)

clf = AdaBoostClassifier(base_estimator=logistic_base, n_estimators=100, learning_rate=learning_rate)

# Crea un pipeline con StandardScaler y el clasificador
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', clf)
])

# Define StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Inicializa una lista para almacenar los resultados de ROC AUC
roc_auc_scores = []

# Itera sobre los diferentes pliegues (folds)
for train_index, test_index in stratified_kfold.split(x_training_features, y1):
    x_train_fold, x_test_fold = x_training_features.iloc[train_index], x_training_features.iloc[test_index]
    y_train_fold, y_test_fold = y1[train_index], y1[test_index]
    
    # Entrena el modelo
    pipeline.fit(x_train_fold, y_train_fold)
    
    # Calcula las probabilidades
    y_pred_proba = pipeline.predict_proba(x_test_fold)[:, 1]
    
    # Calcula y almacena el ROC AUC
    roc_auc = roc_auc_score(y_test_fold, y_pred_proba)
    roc_auc_scores.append(roc_auc)

# Imprime los resultados de ROC AUC
print(f'ROC AUC scores: {roc_auc_scores}')
print(f'Mean ROC AUC: {sum(roc_auc_scores) / len(roc_auc_scores)}')

C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ROC AUC scores: [0.8188670235992246, 0.7870681232977553, 0.8001378403089836, 0.7914018362691435, 0.7767416382302605]
Mean ROC AUC: 0.7948432923410735


In [111]:

"""
Se intenta aplicar GradientBoostingClassifier:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
import pandas as pd

# Crea un estimador GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(random_state=1234)

# Define los hiperparámetros a probar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Define el pipeline solo con StandardScaler
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', gb_clf)
])

# Define StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Realiza la búsqueda en rejilla con StratifiedKFold y roc_auc como métrica
grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_kfold, scoring='roc_auc')
grid_search.fit(x_training_features.values, y1)

# Obtiene el mejor modelo
best_model = grid_search.best_estimator_

# Obtén las probabilidades (asegúrate de que x_test esté en el mismo formato que x_training_features)
x_test_scaled = grid_search.best_estimator_.named_steps['scaler'].transform(x_test)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

# Calcula el ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'ROC AUC en test: {roc_auc}')

# Los mejores hiperparámetros
best_params = grid_search.best_params_
print(f'Los mejores hiperparámetros son: {best_params}')"""

ValueError: Invalid parameter 'learning_rate' for estimator Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', GradientBoostingClassifier(random_state=1234))]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:
#############################preparando el dataset a predecir##################################
df_test_original=pd.read_csv("test_set_features.csv")
col=pd.read_csv("training_set_features_eda_notnulls.csv").columns.tolist()
col=col[2:]
df_test_original=df_test_original[col]